In [1]:
# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *

import pickle
from captum.attr import *
import random
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

np.set_printoptions(threshold=np.inf)

# Custom modules
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from explain_pointnet import *

In [2]:
# Load dataset
data_path = "/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort"

subject_list = find_subjects_parallel(data_path)

Csv files: ['/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort/idaSearch_3_19_2025.csv']


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

Checking the significance of left and right hippocampi on models run with both

In [4]:
subject = sample(subject_list, 1)[0]

In [5]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [6]:
lr_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_Right-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

lr_attributions, lr_pred_research_group = pointnet_ig(model, lr_cloud, device)

vis_attributions(lr_attributions, subject, lr_cloud, lr_pred_research_group)

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f64067eed20_0&reconnect=auto" class="pyvi…

In [7]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_19-03-2025_16-12-19/run_19-03-2025_16-12-19_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [8]:
l_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

l_attributions, l_pred_research_group = pointnet_ig(model, l_cloud, device)

vis_attributions(l_attributions, subject, l_cloud, l_pred_research_group)

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f6406732180_1&reconnect=auto" class="pyvi…

Interesting experiment comparing attributions from two permutations of the same cloud

In [9]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_15-35-05/run_18-03-2025_15-35-05_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [10]:
cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))
    

In [11]:
attributions_orig, pred_research_group_orig = pointnet_ig(model, cloud, device)

shuffler = np.random.permutation(cloud.shape[0])

unshuffler = np.argsort(shuffler)

cloud_shuffled = np.array([cloud[i] for i in shuffler])

attributions_shuffled, pred_research_group_shuffle = pointnet_ig(model, cloud_shuffled, device)

cloud_unshuffled = np.array([cloud_shuffled[i] for i in unshuffler])

attributions_unshuffled = np.array([attributions_shuffled[i] for i in unshuffler])

attributions_diff = attributions_orig - attributions_unshuffled

# NB can't really visualise attributions as they will be normalsied and look large
print(attributions_diff)

if pred_research_group_orig != pred_research_group_shuffle:
    
    print("Research groups are different after shuffle")

print(attributions_diff.shape)
    

[[ 5.05859112e-09 -4.68164998e-09  1.62468593e-10]
 [ 5.24208287e-08 -1.29034649e-07  2.35532193e-07]
 [-2.10364366e-12 -3.03693925e-12  9.41834454e-12]
 [-2.21590800e-10 -1.01118654e-09  3.41306113e-09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.83907092e-07  4.49383061e-07 -9.09663118e-07]
 [-2.80882916e-08  2.53653166e-09 -1.92469779e-08]
 [ 2.15436720e-07 -3.16811708e-07  1.11289228e-06]
 [ 2.02143396e-10  6.04100212e-10 -2.17747077e-09]
 [ 3.91356474e-08 -6.68737319e-08  1.64909291e-07]
 [-9.89771494e-13  3.67967804e-13 -4.19097966e-13]
 [-5.43023691e-10 -8.12833389e-11 -1.59421159e-10]
 [-1.34323745e-15 -4.09324804e-15  1.07708940e-14]
 [ 3.02939451e-16 -2.90736967e-16  2.90884092e-16]
 [-4.87547910e-08  3.25176674e-08 -1.02914733e-07]
 [ 1.78226641e-07 -5.79781241e-08  1.48900877e-07]
 [ 7.10323610e-08 -1.61463734e-07  6.04539669e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.76872436e-08  7.80594546e-08  8.76024248e-09]
 [ 1.15161803e-08  1.48227402e-

In [12]:
vis_attributions(attributions_orig, subject, cloud, pred_research_group_orig)

# These two should look identical if the method was correct
vis_attributions(attributions_shuffled, subject, cloud_shuffled, pred_research_group_shuffle)

vis_attributions(attributions_unshuffled, subject, cloud_unshuffled, pred_research_group_shuffle)

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f6310ecf4d0_2&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f63100ac380_3&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f6310160170_4&reconnect=auto" class="pyvi…

Comparison between transposing the point cloud before or after passing to integrated gradients and saliency. Treating the points as features rather than the dimensions yields better results:

In [13]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [14]:

attributions, pred_research_group = pointnet_ig(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f64d77749b0_5&reconnect=auto" class="pyvi…

In [15]:

attributions, pred_research_group = pointnet_ig_deprecated(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f6310f04b90_6&reconnect=auto" class="pyvi…

In [16]:

attributions, pred_research_group = pointnet_saliency(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/.venv/lib/python3.12/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f630974a9c0_7&reconnect=auto" class="pyvi…

In [17]:

attributions, pred_research_group = pointnet_saliency_deprecated(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/.venv/lib/python3.12/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Widget(value='<iframe src="http://localhost:43733/index.html?ui=P_0x7f63101633b0_8&reconnect=auto" class="pyvi…